In [ ]:
import json

from graph_opt.graph_retrieval import query_entities_before_date
from llm.factory import LLMInterface
from utils.json_utils import extract_json


last_mofication_datetime = "2025-05-26 03:40:00"
llm_client = LLMInterface(
    "bedrock", "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
)

def get_query_to_optimize(last_mofication_datetime):
    entities = query_entities_before_date(last_mofication_datetime)
    if not entities:
        return []
    
    prompt = f"""
    You are a helpful assistant that helps to optimize the graph.
    You are given a list of entities that have been modified before the given datetime.
    Please generate at most 10 queries related to the entities, which will be used to retrieve the entities from the graph based on the queries, and let other agents optimize them.

    Here are the entities:
    {entities}

    response in json format(json array and surrounded by ```json):
    ```json
    [
        "query1",
        "query2",
        ...
    ]
    ```
    """

    try:
        response = llm_client.generate(prompt)
        json_str = extract_json(response)
        json_str = "".join(
            char for char in json_str if ord(char) >= 32 or char in "\r\t"
        )
        return json.loads(json_str)
    except Exception as e:
        print(f"Error extracting json: {e}")
        return None

while True:
    queries = get_query_to_optimize(last_mofication_datetime)
    if queries is []:
        print("No queries generated")
        exit()

    if queries is None:
        print("Error generating queries, waiting 60 seconds for next try")
        time.sleep(60)
        continue

    for query in queries:
        improve_graph(query)
    
    # Sleep for 10 minutes before next iteration
    print(f"Sleeping for 1 minutes... Current time: {datetime.now()}")
    time.sleep(60)  # 600 seconds = 10 minutes


Found 10 entities modified before 2025-05-26 03:40:00


['What is the LOCATE function in SQL and how is it used?',
 'How does the high-concurrency parameter affect TiKV performance?',
 'What security practices should be implemented for the Root User in TiDB?',
 'What are the different sources for binding creation in TiDB?',
 'Why was the lc_time_names system variable changed to read-only in TiDB 5.4.0?',
 'How does Manual Binding Creation differ from Automatic Binding Capture?',
 'What is the purpose of Automatic Binding Evolution in TiDB?',
 "How are Schema IDs used in TiDB's architecture?",
 'What is a Secondary Database in the context of TiDB Binlog?',
 "How do TiDB's thread pools manage high-priority read requests?"]

In [ ]:
import requests
from datetime import datetime, timedelta
from typing import List
import json

from optimization import improve_graph
from setting.db import SessionLocal


upper_datetime = "2025-05-26 03:40:00"

sql = f"""SELECT name, description
FROM entities_210001
WHERE last_modified_at < "{upper_datetime}"
ORDER BY last_modified_at ASC
LIMIT 10;"""



def get_task_goals(
    base_url: str = "https://stackvm.tidb.ai/",
   last_hours: int = 2
) -> List[str]:
    try:
        end_time = datetime.utcnow()
        start_time = end_time - timedelta(hours=last_hours)
        
        url = f"{base_url}/api/tasks/evaluation"
        params = {
            "start_time": start_time.isoformat(),
            "end_time": end_time.isoformat()
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        tasks = response.json()
        
        goals = [task.get("goal", "") for task in tasks if task.get("goal")]
        
        return goals
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching task goals: {e}")
        return []
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return []
    except Exception as e:
        print(f"Unknown error: {e}")
        return []
import time

while True:
    goals = get_task_goals()
    for goal in goals:
        improve_graph(goal)
    
    # Sleep for 10 minutes before next iteration
    print(f"Sleeping for 10 minutes... Current time: {datetime.now()}")
    time.sleep(600)  # 600 seconds = 10 minutes

/var/folders/p0/740z91x53hx0_mmy03yymrzh0000gp/T/ipykernel_65862/2387476037.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow()


Found new issues 0, total issues 221
Identified 217 valid issues
issue is resolved {('redundancy_entity', (33346, 363328, 1380022, 1380024, 1410020, 1500022)): True, ('redundancy_entity', (35666, 660260, 1500021)): True, ('redundancy_entity', (50205, 361880, 1380027, 1440021, 1500020)): True, ('redundancy_entity', (30178, 1500023)): True, ('redundancy_entity', (67446, 361880, 1410021)): True, ('redundancy_entity', (41603, 59518, 900158)): True, ('redundancy_entity', (34673, 50555, 53518, 53534)): True, ('redundancy_entity', (58191, 900373)): True, ('entity_quality_issue', (241263,)): True, ('entity_quality_issue', (43094,)): True, ('entity_quality_issue', (41041,)): True, ('entity_quality_issue', (50555,)): True, ('entity_quality_issue', (57765,)): True, ('redundancy_entity', (32646, 34144, 39735, 51787, 67835, 243832, 720293)): True, ('redundancy_entity', (36675, 240202)): True, ('redundancy_entity', (34877, 59951)): True, ('redundancy_relationship', (34335, 40987, 43623, 43642, 24074

2025-05-24 23:00:58 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Merged relationship created with ID: 1320091 -> 363101(930015)
Deleted 2 relationships


2025-05-24 23:00:58 - graph_search.concrete_search - INFO - Starting search with query: tidb region size 由96M调整到256M，会带来哪些性能优化?


...

Success update entity(4) 37005 to {'name': 'Object Relational Mapping (ORM)', 'description': "A programming technique that bridges the gap between incompatible type systems in object-oriented programming languages and relational databases. In the TiDB ecosystem, ORM functionality can be leveraged as an alternative migration strategy for data re-deployment and migration between database systems. Unlike other migration methods like Dumpling with TiDB Lightning or Data Migration (DM), ORM-based migration utilizes the application's existing object-relational mapping layer to facilitate the transfer of data to TiDB, potentially reducing downtime during migration processes.", 'meta': {'description': 'Use ORM functionality for data re-deployment and migration.', 'topic': 'Alternative Migration Strategy', 'use_case': 'Database migration to TiDB', 'relationship_to_tidb': 'Alternative to standard TiDB migration tools', 'migration_context': 'Can be used when migrating from systems like MariaDB to

2025-05-25 01:03:17 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-25 01:03:17 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Merged entity(('redundancy_entity', (44867, 243103))) created with ID: Resolved timestamp (resolved-ts)(1920026)
Relationships updated for merged entity(('redundancy_entity', (44867, 243103))) 1920026
Merged entity(('redundancy_entity', (44867, 243103))) processing complete.
merged entity(('redundancy_entity', (55009, 63589, 360857, 360859, 362892))) {'name': '-h, --help', 'description': "The '-h, --help' option is a universal command-line flag available across TiUP, TiDB Cloud CLI, and related tools that displays help information for the specified command. When enabled, it prints usage details, available options, arguments, and other guidance to assist users in understanding command functionality. This option is typically disabled by default (with a Boolean data type and false value), and can be enabled by adding it to a command with either no value or a true value. The help information includes command syntax, available flags, usage examples, and other relevant details that vary depe

2025-05-25 01:03:20 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-05-25 01:03:21 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Merged entity(('redundancy_entity', (55009, 63589, 360857, 360859, 362892))) created with ID: -h, --help(1920027)
Relationships updated for merged entity(('redundancy_entity', (55009, 63589, 360857, 360859, 362892))) 1920027
Merged entity(('redundancy_entity', (55009, 63589, 360857, 360859, 362892))) processing complete.
pendding relationship quality issues number 0
pendding redundancy relationships number 1
start to merge relationships ('redundancy_relationship', (33085, 33101, 37870, 37943, 38262, 38721, 38968, 40594, 42284, 42340, 42415, 42798, 47040, 47918, 57099, 57346, 57434, 57494, 90551, 93357, 241461, 241658, 241874, 244613, 270254, 840020)) for {'issue_type': 'redundancy_relationship', 'affected_ids': [47040, 38721, 57346, 244613, 57099, 33101, 40594, 241874, 840020, 57494, 57434, 42340, 42284, 93357, 47918, 270254, 42415, 42798, 37870, 241461, 37943, 90551, 38968, 38262, 241658, 33085], 'reasoning': 'Numerous relationships (IDs: 270254, 47918, 47040, 38262, 38968, 241658, 24